In [13]:
import pandas as pd
import numpy as np
import urllib.request as libreq
from groq import Groq
import xml.etree.ElementTree as ET
import certifi
import os
from datetime import datetime
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()
from bs4 import BeautifulSoup
import re
import PyPDF2
import io
import warnings
warnings.filterwarnings('ignore')
from scholarly import ProxyGenerator

os.environ['SSL_CERT_FILE'] = certifi.where()

In [2]:
pdf_link = 'arxiv.org/pdf/2503.01945'
pdf_response = libreq.urlopen('https://' + pdf_link)
pdf_file = pdf_response.read()
pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_file))

In [3]:
# Extract text from PDF
pdf_text = ''
for page in pdf_reader.pages:
    pdf_text += page.extract_text()

In [4]:
pdf_text

'Astronomy &Astrophysics manuscript no. aanda ©ESO 2025\nMarch 5, 2025\nNo [CII] or dust detection in two Little Red Dots at zspec>7\nMengyuan Xiao1⋆, Pascal A. Oesch1,2,3, Longji Bing4, David Elbaz5, Jorryt Matthee6, Yoshinobu Fudamoto7,8, Seiji\nFujimoto9,10,11, Rui Marques-Chaves1, Christina C. Williams12,8, Miroslava Dessauges-Zavadsky1, Francesco\nValentino2,13, Gabriel Brammer2,3, Alba Covelo-Paz1, Emanuele Daddi5, Johan P. U. Fynbo2,3, Steven Gillman2,13,\nMichele Ginolfi14,15, Emma Giovinazzo1, Jenny E. Greene16, Qiusheng Gu17,18, Garth Illingworth19, Kohei Inayoshi20,\nVasily Kokorev10, Romain A. Meyer1, Rohan P. Naidu21,9, Naveen A. Reddy22, Daniel Schaerer1, Alice Shapley23,\nMauro Stefanon24,25, Charles L. Steinhardt26, David J. Setton16, Marianne Vestergaard27,8, and Tao Wang17,18\n(Affiliations can be found after the references)\nReceived xxx; accepted xxx\nABSTRACT\nLittle Red Dots (LRDs) are compact, point-like sources characterized by their red color and broad Balmer l

In [5]:
# Author Names 
# Extract all text till the word ABSTRACT appears
abstract_index = pdf_text.find('ABSTRACT')
text_till_abstract = pdf_text[:abstract_index]
text_till_abstract

'Astronomy &Astrophysics manuscript no. aanda ©ESO 2025\nMarch 5, 2025\nNo [CII] or dust detection in two Little Red Dots at zspec>7\nMengyuan Xiao1⋆, Pascal A. Oesch1,2,3, Longji Bing4, David Elbaz5, Jorryt Matthee6, Yoshinobu Fudamoto7,8, Seiji\nFujimoto9,10,11, Rui Marques-Chaves1, Christina C. Williams12,8, Miroslava Dessauges-Zavadsky1, Francesco\nValentino2,13, Gabriel Brammer2,3, Alba Covelo-Paz1, Emanuele Daddi5, Johan P. U. Fynbo2,3, Steven Gillman2,13,\nMichele Ginolfi14,15, Emma Giovinazzo1, Jenny E. Greene16, Qiusheng Gu17,18, Garth Illingworth19, Kohei Inayoshi20,\nVasily Kokorev10, Romain A. Meyer1, Rohan P. Naidu21,9, Naveen A. Reddy22, Daniel Schaerer1, Alice Shapley23,\nMauro Stefanon24,25, Charles L. Steinhardt26, David J. Setton16, Marianne Vestergaard27,8, and Tao Wang17,18\n(Affiliations can be found after the references)\nReceived xxx; accepted xxx\n'

In [18]:
pg = ProxyGenerator()
pg.FreeProxies()

client_summary = Groq(
    api_key=os.environ['GROQ_API_KEY_1'],
)

content = text_till_abstract
summary = client_summary.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You need to extract the author names from the text.",
        },
        
        {
            "role": "user",
            "content": content,
        }
    ],
    model="llama3-8b-8192",
    stream=False,
)

NotFoundError: Error code: 404 - {'error': {'message': 'Not Found'}}

In [19]:
import requests

url = "http://chese.vercel.app/api"
payload = {
    "prompt": "Hello from Python!"
}

response = requests.post(url, json=payload)
print("Status Code:", response.status_code)
print("Response Data:", response.json())

Status Code: 200
Response Data: {'data': "Hello to you too! Python is a versatile and widely-used programming language. Here's a bit about it:\n\n*   **What it is:** Python is a high-level, general-purpose programming language known for its readability. It emphasizes code readability through the use of indentation.\n*   **Who created it:** It was created by Guido van Rossum and released in 1991.\n*   **What it's used for:** Python is used in various applications, including:\n\n    *   Web development (server-side)\n    *   Software development\n    *   Data science and machine learning\n    *   Data analysis\n    *   Task automation\n    *   System scripting\n*   **Key features:**\n\n    *   **Easy to learn:** Python has a simple syntax, similar to the English language.\n    *   **Versatile:** It can be used for various programming paradigms like procedural, object-oriented, and functional programming.\n    *   **Cross-platform:** Python works on different platforms like Windows, Mac, 